In [23]:
%pip install setuptools
%pip install ydata_profiling
%pip install scikit-learn
%pip install xlrd
%pip install openpyxl
%pip install ipywidgets
%pip install statsmodels
%pip install catboost

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from ydata_profiling import ProfileReport
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from statsmodels.tsa.stattools import adfuller
from catboost import CatBoostRegressor
import catboost
from catboost import * 

In [25]:
excel_data_volumes=pd.read_excel("././dataset/Объёмы перевозок.xls", skiprows=[0], header=[0,1])

In [26]:
excel_data_volumes.sort_index(axis=1, inplace=True)
num_col=len(excel_data_volumes.columns)
left = excel_data_volumes.iloc[:, [num_col-5, num_col-4, num_col-3, num_col-2,num_col-1]]
left.columns = left.columns.get_level_values(1)
left = left.set_axis(["id", "subj_from", "subj_to", "cargo_code", "cargo_type"], axis = 1)

In [27]:
labelencodder = LabelEncoder()
subj = pd.concat([left.loc[:, ["subj_from"]].rename(columns = {"subj_from": "subj"}), left.loc[:, ["subj_to"]].rename(columns = {"subj_to": "subj"})]).drop_duplicates(ignore_index=True)
cargo_type = left.loc[:, ["cargo_type"]].drop_duplicates(ignore_index=True)

In [28]:
left.loc[:, ["subj_from"]] = left.loc[:, ["subj_from"]].apply(lambda x: subj[subj["subj"] == x["subj_from"]].index[0], axis=1)
left.loc[:, ["subj_to"]] = left.loc[:, ["subj_to"]].apply(lambda x: subj[subj["subj"] == x["subj_to"]].index[0], axis=1)
left.loc[:, ["cargo_type"]] = left.loc[:, ["cargo_type"]].apply(lambda x: cargo_type[cargo_type["cargo_type"] == x["cargo_type"]].index[0], axis=1)

In [29]:
left["subj_from"] = left["subj_from"].astype("int64")
left["subj_to"] = left["subj_to"].astype("int64")
left["cargo_type"] = left["cargo_type"].astype("int64")

In [30]:
for i in range(int((num_col-5)/2)):
    right = excel_data_volumes.iloc[:, [i*2,i*2+1]]
    date = pd.to_datetime(excel_data_volumes.iloc[:, [i*2]].columns.get_level_values(0).to_list()[0], format="%Y/%m")
    right.insert(2, ("", "date"), date)
    right.insert(3, ("", "month"), date.month)
    right.insert(4, ("", "quarter"), date.quarter)
    right.insert(5, ("", "year"), date.year)
    right.columns = right.columns.get_level_values(1)
    right = right.set_axis(["cargo_volume", "cargo_sales", "date", "month", "quarter", "year"], axis=1)
    if i == 0:
        result = pd.concat([left, right], axis = 1, sort = False)
    else:
        result = pd.concat([result, pd.concat([left, right], axis = 1, sort = False)], axis = 0, ignore_index=True, sort=False)
excel_data_volumes = result

In [31]:
excel_data_volumes.head()

,id,subj_from,subj_to,cargo_code,cargo_type,cargo_volume,cargo_sales,date,month,quarter,year
0,21220,0,12,39146,0,0,0,2022-01-01,1,1,2022
1,912,1,13,32411,1,0,0,2022-01-01,1,1,2022
2,912,1,14,32411,1,476,1440050,2022-01-01,1,1,2022
3,912,1,6,32411,1,0,0,2022-01-01,1,1,2022
4,912,1,12,32411,1,0,0,2022-01-01,1,1,2022


In [32]:
excel_data_volumes.insert(11, ("month_of_period"), 0)

In [33]:
min = excel_data_volumes["year"].min()
excel_data_volumes["month_of_period"] = excel_data_volumes["year"].apply(lambda x: x - min)*12 + excel_data_volumes["month"] 
excel_data_volumes = excel_data_volumes.drop("date", axis = 1)

In [34]:
excel_data_volumes.head(10)

,id,subj_from,subj_to,cargo_code,cargo_type,cargo_volume,cargo_sales,month,quarter,year,month_of_period
0,21220,0,12,39146,0,0,0,1,1,2022,1
1,912,1,13,32411,1,0,0,1,1,2022,1
2,912,1,14,32411,1,476,1440050,1,1,2022,1
3,912,1,6,32411,1,0,0,1,1,2022,1
4,912,1,12,32411,1,0,0,1,1,2022,1
5,912,1,12,32102,1,0,0,1,1,2022,1
6,912,1,5,32411,1,808,928027,1,1,2022,1
7,912,1,15,32411,1,131,216406,1,1,2022,1
8,912,1,16,32411,1,454,692989,1,1,2022,1
9,912,1,11,32411,1,475,559474,1,1,2022,1


In [35]:
excel_data_volumes = excel_data_volumes.drop("subj_from", axis = 1)
excel_data_volumes = excel_data_volumes.drop("subj_to", axis = 1)
excel_data_volumes = excel_data_volumes.drop("cargo_code", axis = 1)
excel_data_volumes = excel_data_volumes.drop("cargo_type", axis = 1)


In [36]:
data_pd = excel_data_volumes.groupby(["id", "month_of_period", "year", "quarter", "month" ], as_index=False).sum()

In [37]:
data_pd["innactive"] = 0
data_pd["active"] = 0
data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd["active"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] > 0) & (x["cargo_sales"] > 0)) else 0, axis=1)

In [38]:
data_pd.head(10)

,id,month_of_period,year,quarter,month,cargo_volume,cargo_sales,innactive,active
0,9,1,2022,1,1,0,0,1,0
1,9,2,2022,1,2,0,0,1,0
2,9,3,2022,1,3,0,0,1,0
3,9,4,2022,2,4,0,0,1,0
4,9,5,2022,2,5,0,0,1,0
5,9,6,2022,2,6,0,0,1,0
6,9,7,2022,3,7,0,0,1,0
7,9,8,2022,3,8,0,0,1,0
8,9,9,2022,3,9,61,155364,0,1
9,9,10,2022,4,10,0,0,1,0


In [39]:
data_pd["outflow"] = 0

def calc_innactive(row):
    prev_row_value = 0
    row_value = data_pd.iloc[row.name]["innactive"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["innactive"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("innactive")] = prev_row_value + 1
                row_value = data_pd.iloc[row.name]["innactive"]
                if row_value > 11:
                    prev_row_value = 1
                else:
                    prev_row_value = row_value/12
            else: 
                prev_row_value = 0
    return prev_row_value

def calc_active(row):
    row_value = data_pd.iloc[row.name]["active"]
    row_id = data_pd.iloc[row.name]["id"]
    if row.name > 0:
        prev_row_id = data_pd.iloc[row.name - 1]["id"]
        if row_id == prev_row_id:
            if  row_value > 0:
                prev_row_value = data_pd.iloc[row.name - 1]["active"]
                data_pd.iloc[row.name, data_pd.columns.get_loc("active")] = prev_row_value + 1
    return 0

data_pd["outflow"] = data_pd.apply(calc_innactive, axis=1)
# data_pd["innactive"] = data_pd.apply(lambda x: 1 if ((x["cargo_volume"] == 0) & (x["cargo_sales"] == 0)) else 0, axis=1)
data_pd.apply(calc_active, axis=1)
data_pd["cargo_price"] = data_pd.apply(lambda x: (x["cargo_sales"] / x["cargo_volume"]) if x["cargo_volume"] > 0 else x["cargo_sales"], axis= 1)
data_pd["out"] = data_pd.apply(lambda x: 1 if (x["outflow"] == 1) else 0, axis=1).astype("int64")


In [40]:
data_pd.head(10)

,id,month_of_period,year,quarter,month,cargo_volume,cargo_sales,innactive,active,outflow,cargo_price,out
0,9,1,2022,1,1,0,0,1,0,0.000000,0.00000,0
1,9,2,2022,1,2,0,0,2,0,0.166667,0.00000,0
2,9,3,2022,1,3,0,0,3,0,0.250000,0.00000,0
3,9,4,2022,2,4,0,0,4,0,0.333333,0.00000,0
4,9,5,2022,2,5,0,0,5,0,0.416667,0.00000,0
5,9,6,2022,2,6,0,0,6,0,0.500000,0.00000,0
6,9,7,2022,3,7,0,0,7,0,0.583333,0.00000,0
7,9,8,2022,3,8,0,0,8,0,0.666667,0.00000,0
8,9,9,2022,3,9,61,155364,0,1,0.000000,2546.95082,0
9,9,10,2022,4,10,0,0,1,0,0.083333,0.00000,0


In [41]:
keep = data_pd['id'].drop_duplicates().sample(frac=0.7)
m = data_pd['id'].isin(keep)
train = data_pd[m]
test = data_pd[~m]

train.to_excel("train.xlsx")
test.to_excel("test.xlsx")